**<font size="6">Módulo de máscara-recorte</font>**

<dl>
    <dt>La primera parte carga el módulo obtenido en LV_segmentation.ipynb y lo usa en todas las imágenes</dt>
    <dt>La segunda parte recorta las imágenes originales usando la máscara obtenida</dt>
</dl>

In [2]:
# Program: <Mascara_recorte_all.ipynb>
# Author: Marc Bracons Cucó
# Affiliation: Department of Telecommunication and Systems Engineering, Autonomous University of Barcelona, Wireless Information Networking Group
# Copyright © 2023 Marc Bracons Cucó
# 
# This program is proprietary software; you may not use, distribute, or modify it 
# without the explicit permission of the author.
#
# If you wish to use this program in your work, please, contact the author.

In [2]:
#Obtención de las máscaras -> ~14300 imágenes

**<font size="4">Primera parte: Obtención de máscaras</font>**
<dl>
Aproximadamente 14300 imágenes
</dl>

In [3]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Cargar el modelo previamente guardado
model = load_model('/home/win001/00_heart/00_Dataset_CNN/02_LV_CNN/unet_model_100.h5')

# Directorios de entrada
root_path = "/home/win001/00_heart/00_Dataset_CNN/02_ALL_FRAMES"
categories = ["MCH", "Otros"]

# Procesar todas las imágenes en el dataset
for category in categories:
    category_path = os.path.join(root_path, category)
    for patient_folder in os.listdir(category_path):
        patient_path = os.path.join(category_path, patient_folder)
        for frame_folder in os.listdir(patient_path):
            frame_path = os.path.join(patient_path, frame_folder)

            # Procesar todas las imágenes en el directorio de entrada
            for img_name in os.listdir(frame_path):
                img_path = os.path.join(frame_path, img_name)

                # Cargar la imagen
                test_img = tf.io.decode_png(tf.io.read_file(img_path), channels=1)
                test_img = tf.cast(test_img, tf.float32) / 255.0
                test_img = tf.image.resize(test_img, (256, 256))

                # Obtener la predicción del modelo
                pred_mask = model.predict(np.array([test_img]))

                # Guardar la máscara predicha en la misma carpeta que la imagen original
                output_directory = os.path.dirname(img_path)
                output_filename = "mask_" + os.path.basename(img_path)
                output_path = os.path.join(output_directory, output_filename)

                pred_mask_img = (pred_mask.squeeze() * 255).astype(np.uint8)
                plt.imsave(output_path, pred_mask_img, cmap="gray")

2023-05-08 09:59:33.537164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 286ms/step


**<font size="4">Segunda parte: Recorte de imágenes</font>**

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Directorios de entrada
root_path = "/home/win001/00_heart/00_Dataset_CNN/02_ALL_FRAMES"
categories = ["MCH", "Otros"]

# Procesar todas las imágenes en el dataset
for category in categories:
    category_path = os.path.join(root_path, category)
    for patient_folder in os.listdir(category_path):
        patient_path = os.path.join(category_path, patient_folder)
        for frame_folder in os.listdir(patient_path):
            frame_path = os.path.join(patient_path, frame_folder)

            # Directorio de salida
            output_dir = os.path.join(frame_path, "recortes")
            os.makedirs(output_dir, exist_ok=True)

            # Procesar todas las imágenes en el directorio de entrada
            for img_name in os.listdir(frame_path):
                if not img_name.startswith("mask_") and img_name.endswith((".png", ".jpg")):
                    img_path = os.path.join(frame_path, img_name)
                    mask_path = os.path.join(frame_path, "mask_" + img_name)

                    # Cargar la imagen original y la máscara
                    img = Image.open(img_path).convert("RGBA")
                    mask = Image.open(mask_path).convert("L")

                    # Redimensionar la máscara para que coincida con las dimensiones de la imagen original
                    mask = mask.resize(img.size)

                    # Umbralizar la máscara para crear una máscara binaria
                    threshold = 128
                    mask = mask.point(lambda p: p > threshold and 255)

                    # Aplicar la máscara binaria a la imagen original
                    img.putalpha(mask)

                    # Guardar el recorte en la carpeta "recortes"
                    output_filename = "recorte_" + img_name
                    output_path = os.path.join(output_dir, output_filename)

                    img.save(output_path)